In [1]:
import json
import os
import ase.io

from ase.io                       import write
from ase.io.vasp                  import read_vasp
from ase.build                    import bulk, sort
from ase.calculators.emt          import EMT
from hiphive.structure_generation import generate_mc_rattled_structures

Read structures containing displacements and forces

In [2]:
root_path = f'data/CrN-riccardo/PM/SC_2x2x2_Vol_76.69'
#root_path = f'data/CrN-riccardo/PM/SC_2x2x2_Vol_76.31'
#root_path = f'data/CrN-riccardo/PM/SC_2x2x2_Vol_75.93'
#root_path = f'data/CrN-riccardo/PM/SC_2x2x2_Vol_75.56'

# Parameters
n_structures     = 500
rattle_std       = 0.03
minimum_distance = 1.5
cell_size        = [2, 2, 2]

# Create and save as a dictionary
setup_variables = {
    'root_path':        root_path,
    'n_structures':     n_structures,
    'rattle_std':       rattle_std,
    'minimum_distance': minimum_distance,
    'cell_size':        cell_size
}

# Write the dictionary to the file in JSON format
with open(f'{root_path}/structure_generation_variables.json', 'w') as json_file:
    json.dump(setup_variables, json_file)

In [3]:
# Read POSCAR file
prim = read_vasp(file=f'{setup_variables["root_path"]}/POSCAR')
atoms_ideal = prim.repeat(cell_size)

# Generate the structures
structures = generate_mc_rattled_structures(atoms_ideal, n_structures, rattle_std, minimum_distance)

# Save the structures for later use in force determination
#write(f'{setup_variables["root_path"]}/prim.extxyz',                 prim)  # Primitive cell
#write(f'{setup_variables["root_path"]}/supercell_ideal_prev.extxyz', atoms_ideal)  # Ideal supercell
#write(f'{setup_variables["root_path"]}/supercells_rattled.extxyz',   structures)

In [4]:
for idx in range(n_structures):
    # Sort the chemical species
    structure = sort(structures[idx])
    
    # Create disp folder
    idx_path = f'{setup_variables["root_path"]}/disp-{idx}'
    os.mkdir(idx_path)
    
    # Write to POSCAR
    write(f'{idx_path}/POSCAR', structure, format='vasp')